<a href="https://colab.research.google.com/github/jiobu1/DS-Unit-2-Applied-Modeling/blob/master/Jisha_Obukwelu_DSPT3_231_Applied_Modeling_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files
upload = files.upload()

Saving states_all.csv to states_all.csv


###***Choose your target. Which column in your tabular dataset will you predict?***

This project will predict state's passing rate on the naep assessment based on features from the dataset. 

###***Is your problem regression or classification?***

This is a classification problem, I will be engineering the classification column based on the cut scores (score range) from the NAEP assessment for basic, proficient, and advanced. I will have to break this down further because the classes are heavily imbalanced. 

###***How is your target distributed?***
Classification: How many classes? Are the classes imbalanced?

In [0]:
import pandas as pd
import numpy as np

In [0]:
all_states = pd.read_csv('states_all.csv')

In [408]:
all_states.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [409]:
all_states.shape

(1492, 25)

There should be three classes (that I will engineer) but the classes are gravely imbalanced with one class not having any values in it. Going to try and separate the common class to try and help with the imbalance. 

###***Choose your evaluation metric(s).***
Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?

In [410]:
target_df = all_states[['PRIMARY_KEY','AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE' ]].copy()
target_df.head()

,PRIMARY_KEY,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,208.398961,260.892247,196.764414,NaN


Separating columns necessary to create new target columns. 

####4TH GRADE READING

In [411]:
target_df['AVG_READING_4_SCORE'] = target_df['AVG_READING_4_SCORE'].fillna(0)
target_df['AVG_READING_4_SCORE'] = target_df['AVG_READING_4_SCORE'].astype(int)
target_df['AVG_READING_4_SCORE']

0       207
1         0
2       206
3       208
4       196
       ... 
1487    227
1488    223
1489    217
1490    219
1491    226
Name: AVG_READING_4_SCORE, Length: 1492, dtype: int64

In [412]:
def proficiency(row):
    if 0 < row['AVG_READING_4_SCORE'] < 238:
        val = 'basic'
    elif 238 <= row['AVG_READING_4_SCORE'] < 268:
        val = 'proficient'
    elif row['AVG_READING_4_SCORE'] >= 268:
        val = 'advanced'
    else:
        val = 0
    return val

target_df['READING_PROF_4'] = target_df.apply(proficiency, axis=1)
target_df['READING_PROF_4']

0       basic
1           0
2       basic
3       basic
4       basic
        ...  
1487    basic
1488    basic
1489    basic
1490    basic
1491    basic
Name: READING_PROF_4, Length: 1492, dtype: object

In [413]:
min(target_df['AVG_READING_4_SCORE']), max(target_df['AVG_READING_4_SCORE'])

(0, 236)

So this won't work - It's is 100% basic. Let me see if I can use another one of the score columns for predictions.




####4TH GRADE MATH 

In [414]:
min(target_df['AVG_MATH_4_SCORE']), max(target_df['AVG_MATH_4_SCORE'])

(187.134669958413, 253.420961133202)

In [415]:
target_df['AVG_MATH_4_SCORE'] = target_df['AVG_MATH_4_SCORE'].fillna(0)
target_df['AVG_MATH_4_SCORE'] = target_df['AVG_MATH_4_SCORE'].astype(int)
target_df['AVG_MATH_4_SCORE']

0       208
1         0
2       215
3       210
4       208
       ... 
1487    248
1488    241
1489    236
1490    240
1491    247
Name: AVG_MATH_4_SCORE, Length: 1492, dtype: int64

In [416]:
def proficiency(row):
    if 0 < row['AVG_MATH_4_SCORE'] < 249:
        val = 'basic'
    elif 249 <= row['AVG_MATH_4_SCORE'] < 282:
        val = 'proficient'
    elif row['AVG_MATH_4_SCORE'] >= 282:
        val = 'advanced'
    else:
        val = 0
    return val

target_df['MATH_PROF_4'] = target_df.apply(proficiency, axis=1)
target_df['MATH_PROF_4']

0       basic
1           0
2       basic
3       basic
4       basic
        ...  
1487    basic
1488    basic
1489    basic
1490    basic
1491    basic
Name: MATH_PROF_4, Length: 1492, dtype: object

In [417]:
target_df['MATH_PROF_4'].value_counts()

0             956
basic         522
proficient     14
Name: MATH_PROF_4, dtype: int64

In [418]:
target_df['AVG_MATH_4_SCORE'].value_counts()

0      956
240     35
243     30
237     29
241     29
242     25
238     24
235     23
236     23
239     21
230     19
231     18
244     18
245     17
246     16
232     16
234     14
229     14
228     12
227     11
226     11
233     10
215     10
224      9
247      8
217      8
213      7
220      7
219      7
225      7
222      6
248      6
249      5
223      5
221      4
210      3
208      3
218      3
252      3
253      3
214      2
212      2
211      2
251      2
209      2
204      2
250      1
201      1
193      1
192      1
187      1
Name: AVG_MATH_4_SCORE, dtype: int64

This would not work either but I am wondering why my function did not pick up on the 6 proficient score?
- got it to work now, made a silly mistake. 

####8TH GRADE READING

In [419]:
target_df['AVG_READING_8_SCORE'] = target_df['AVG_READING_8_SCORE'].fillna(0)
target_df['AVG_READING_8_SCORE'] = target_df['AVG_READING_8_SCORE'].astype(int)
target_df['AVG_READING_8_SCORE']

0         0
1       258
2       262
3       264
4         0
       ... 
1487    273
1488    267
1489    271
1490    258
1491    269
Name: AVG_READING_8_SCORE, Length: 1492, dtype: int64

In [420]:
target_df['AVG_READING_8_SCORE'].value_counts()

0      994
267     47
264     37
266     36
268     34
269     32
265     31
260     22
262     21
270     21
261     20
257     19
258     18
259     18
263     15
272     14
251     13
255     12
253     11
254     10
271      9
273      9
256      8
252      7
274      7
250      6
275      3
277      3
276      2
247      2
242      2
238      2
249      1
248      1
246      1
240      1
239      1
236      1
280      1
Name: AVG_READING_8_SCORE, dtype: int64

Instead of separating out the into 3 classiications, I will separate the first class into 3 stages to see if the information is still imbalanced. 

In [421]:
min(target_df['AVG_READING_8_SCORE']), max(target_df['AVG_READING_8_SCORE'])

(0, 280)

In [422]:
def proficiency(row):
  #separating basic into 3 groups (0-281)
    if 0< row['AVG_READING_8_SCORE'] < 260:
        val = 'low-basic'
    elif 260 <= row['AVG_READING_8_SCORE'] < 270:
        val = 'mid-basic'
    elif 270 <= row['AVG_READING_8_SCORE'] < 281:
        val = 'high-basic'
    elif 281 <= row['AVG_READING_8_SCORE'] < 323:
        val = 'proficient'
    elif row['AVG_READING_8_SCORE'] >= 323:
        val = 'advanced'
    else:
        val = 0
    return val

target_df['READING_PROF_8'] = target_df.apply(proficiency, axis=1)
target_df['READING_PROF_8'].astype('str')

0                0
1        low-basic
2        mid-basic
3        mid-basic
4                0
           ...    
1487    high-basic
1488     mid-basic
1489    high-basic
1490     low-basic
1491     mid-basic
Name: READING_PROF_8, Length: 1492, dtype: object

In [423]:
target_df['READING_PROF_8'].value_counts()

0             994
mid-basic     295
low-basic     134
high-basic     69
Name: READING_PROF_8, dtype: int64

* So this won't work either - It's is 100% basic. Let me see if I 
can use another one of the score columns for predictions.
* ***Breaking down basic column***



####8TH GRADE MATH

In [424]:
min(target_df['AVG_MATH_8_SCORE']), max(target_df['AVG_MATH_8_SCORE'])

(232.831509532896, 300.568235011147)

In [425]:
target_df['AVG_MATH_8_SCORE'].value_counts()

253.595187    2
281.035488    1
292.321817    1
283.953345    1
270.415766    1
             ..
276.689980    1
268.432355    1
272.379907    1
267.863726    1
273.978787    1
Name: AVG_MATH_8_SCORE, Length: 531, dtype: int64

In [426]:
target_df['AVG_MATH_8_SCORE'] = target_df['AVG_MATH_8_SCORE'].fillna(0)
target_df['AVG_MATH_8_SCORE'] = target_df['AVG_MATH_8_SCORE'].astype(int)
target_df['AVG_MATH_8_SCORE']

0       252
1         0
2       265
3       256
4       260
       ... 
1487    290
1488    289
1489    273
1490    288
1491    288
Name: AVG_MATH_8_SCORE, Length: 1492, dtype: int64

In [427]:
def proficiency(row):
    if 0 < row['AVG_MATH_8_SCORE'] < 270:
        val = 'low basic'
    elif 270 <= row['AVG_MATH_8_SCORE'] < 280:
        val = 'mid basic '
    elif 280 <= row['AVG_MATH_8_SCORE'] < 299:
        val = 'high basic'    
    elif 299 <= row['AVG_MATH_8_SCORE'] < 333:
        val = 'proficient'
    elif row['AVG_MATH_8_SCORE'] >= 333:
        val = 'advanced'
    else:
        val = 0
    return val

target_df['MATH_PROF_8'] = target_df.apply(proficiency, axis=1)
target_df['MATH_PROF_8'].astype('str')

0        low basic
1                0
2        low basic
3        low basic
4        low basic
           ...    
1487    high basic
1488    high basic
1489    mid basic 
1490    high basic
1491    high basic
Name: MATH_PROF_8, Length: 1492, dtype: object

In [428]:
target_df['MATH_PROF_8'].value_counts()

0             960
high basic    276
mid basic     156
low basic      99
proficient      1
Name: MATH_PROF_8, dtype: int64

* This would not work either but I am wondering why my function did not pick up on the lone proficient score?
* Fixed this as well. 


###***Targets***

Adding the two new engineered features back into the database.

In [429]:
all_states[['READING_PROF_8', 'MATH_PROF_8']] = target_df[['READING_PROF_8','MATH_PROF_8' ]].copy()
all_states.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,READING_PROF_8,MATH_PROF_8
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN,0,low basic
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712,low-basic,0
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895,mid-basic,low basic
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665,mid-basic,low basic
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN,0,low basic


In [430]:
all_states.dtypes

PRIMARY_KEY                      object
STATE                            object
YEAR                              int64
ENROLL                          float64
TOTAL_REVENUE                   float64
FEDERAL_REVENUE                 float64
STATE_REVENUE                   float64
LOCAL_REVENUE                   float64
TOTAL_EXPENDITURE               float64
INSTRUCTION_EXPENDITURE         float64
SUPPORT_SERVICES_EXPENDITURE    float64
OTHER_EXPENDITURE               float64
CAPITAL_OUTLAY_EXPENDITURE      float64
GRADES_PK_G                     float64
GRADES_KG_G                     float64
GRADES_4_G                      float64
GRADES_8_G                      float64
GRADES_12_G                     float64
GRADES_1_8_G                    float64
GRADES_9_12_G                   float64
GRADES_ALL_G                    float64
AVG_MATH_4_SCORE                float64
AVG_MATH_8_SCORE                float64
AVG_READING_4_SCORE             float64
AVG_READING_8_SCORE             float64


In [0]:
all_states['READING_PROF_8'] = all_states['READING_PROF_8'].astype(str)
all_states['MATH_PROF_8'] = all_states['MATH_PROF_8'].astype(str)

###***Choose which observations you will use to train, validate, and test your model.***

I would split this dataset by date. That would make the most sense in this context. 
Does not make sense to split since as a classification problem, greater than 70% of the majority class for any of the predictive features is the majority class of basic. 
- will combat this by separting the basic section into a low medium and high group. 

In [0]:
from sklearn.model_selection import train_test_split

In [438]:
train = all_states[all_states['YEAR'] <= 2011]
test = all_states[all_states['YEAR'] > 2011]
train.shape, test.shape

((1161, 27), (331, 27))

In [439]:
min(train['YEAR']), max(train['YEAR'])

(1992, 2011)

In [440]:
min(test['YEAR']), max(test['YEAR'])

(2012, 2017)

In [441]:
train, val = train_test_split(
    train, train_size=0.80, test_size=0.20,
    stratify=train['READING_PROF_8'], 
    random_state=42
)

train.shape, val.shape, test.shape

((928, 27), (233, 27), (331, 27))

###***Begin to clean and explore your data.***

For this dataset, I would delete the years where the NAEP test scores are not available since that does not give anything for me to train/validate/test the data on. The data is from 1992 - 2017. The NAEP only gives information for the years 1992, 1996, 2000, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017. 

In [442]:
all_states.isnull().sum()

PRIMARY_KEY                       0
STATE                             0
YEAR                              0
ENROLL                          263
TOTAL_REVENUE                   212
FEDERAL_REVENUE                 212
STATE_REVENUE                   212
LOCAL_REVENUE                   212
TOTAL_EXPENDITURE               212
INSTRUCTION_EXPENDITURE         212
SUPPORT_SERVICES_EXPENDITURE    212
OTHER_EXPENDITURE               263
CAPITAL_OUTLAY_EXPENDITURE      212
GRADES_PK_G                     173
GRADES_KG_G                     132
GRADES_4_G                      131
GRADES_8_G                      131
GRADES_12_G                     131
GRADES_1_8_G                    131
GRADES_9_12_G                   131
GRADES_ALL_G                    173
AVG_MATH_4_SCORE                956
AVG_MATH_8_SCORE                960
AVG_READING_4_SCORE             959
AVG_READING_8_SCORE             994
READING_PROF_8                    0
MATH_PROF_8                       0
dtype: int64

### ***WRANGLE***

In [451]:
def wrangle(X):
  # Prevent SettingWithCopyWarning
  X = X.copy()

  # When columns have zeros and shouldn't, they are like null values.
  # So we will replace the zeros with nulls, and impute missing values later.
  cols_with_zeros = ['READING_PROF_8', 'MATH_PROF_8']
  for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan) 
  
  #Dropping rows if they do not contain any score data, will not drop if there is just 1 value
  X = X.dropna(axis=0, subset=['AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'], thresh=1)

  #Return the wrangled dataframe
  return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test) 

train.shape, val.shape, test.shape

((319, 27), (84, 27), (153, 27))

I went from having 1492 rows to 556!

###***Begin to choose which features, if any, to exclude. Would some features "leak" future information?***

If I were to use this dataset, I would delete the scores since that would definitely delete information. 
- either Reading or Math. 